##### First try algorithm.

Here we calculate for each node its in-degree and its out-degree. The idea is that a given link (s,t) is more likely to exist if s is frequently a source (high out-degree) and t frequently a target.

First we create a table :

Source | Target | Link (0 or 1) | Degree_out (for Source) | Degree_in (for Target)

This table is created thanks to pandas.

Then, we train a random forest model (scikit learn) to predict the column Link using the others.
Next, we calculate for the test links the degrees and predict the values (0 or 1).

Finally, we export in the correct format the predictions, ready to be uploaded !

## Import

In [1]:
import numpy as np
import csv
import pandas as pd
import time
from sklearn.ensemble import RandomForestRegressor

## Training DataSet

#### Loading DataSet + counting degrees

In [5]:
## I create my main DataFrame df with the training file and rename my columns

df = pd.read_csv('training.txt', sep=" ", header=None)
df.columns = ['Source', 'Target', 'Link']
print(df)

##Creating dfs, the DataFrame of the node considered as Source

# I count for each node its degree as a Source
dfs = df.groupby(by = "Source").sum().drop("Target", axis = 1)
#dfs = dfs.sort_values("Link", ascending = False) #if I want to sort by decreasing degree
dfs = dfs.rename(columns = {"Link" : "Degree_out"})


##Creating dft, the DataFrame of the node considered as Target

# I count for each node its degree as a Target
dft = df.groupby(by = "Target").sum().drop("Source", axis = 1)
#dft = dft.sort_values("Link", ascending = False) #if I want to sort by decreasing degree
dft = dft.rename(columns = {"Link" : "Degree_in"})

        Source  Target  Link
0        10481    5428     1
1         7353   30328     0
2         8627    3547     1
3        10232   21925     1
4         7110    3288     1
5         6866    9656     0
6         1317    1766     0
7         9458   15439     1
8        12447   21216     0
9         5832    2363     1
10       24640    8893     0
11         981    9975     1
12       12410   16669     1
13       26989    9242     1
14       32581    3257     1
15       31691   22764     1
16       28298   31344     0
17        9231   17006     1
18       25805   12008     0
19        3591   27620     0
20       29948   29548     0
21       22593   30169     0
22       13923   14029     1
23        8517   13111     0
24        8019   27120     1
25       25094   30641     1
26       21416   18585     0
27       15797    2021     1
28       23787   20852     0
29       24608     931     1
...        ...     ...   ...
453767   31292   12565     1
453768     409   30835     0
453769    6573

#### Creating the sources and targets table (not used yet)

In [8]:
## Now for each node, I store all its sources and targets in two table

n = dfs.index.max()
src_list_tar = [0]*(n+1) #I can't initialize at [[]]*(n+1) because of copy issues so I create the list at the first encounter
for i in range(n+1):
    src_list_tar[i] = []
m = dft.index.max()
tar_list_src = [0]*(m+1) #I can't initialize at [[]]*(n+1) because of copy issues so I create the list at the first encounter
for i in range(m+1):
    tar_list_src[i] = []

t1 = time.time()
for id, row in df.iterrows():   #going through df line by line
    if row["Link"] == 1:
        src_list_tar[row["Source"]].append(row["Target"])
        tar_list_src[row["Target"]].append(row["Source"])
t2 = time.time()
print("Time : " + str(int(t2-t1)) + "s")

Time : 49s


,Source,Target,Link
0,10481,5428,1
1,7353,30328,0
2,8627,3547,1
3,10232,21925,1
4,7110,3288,1
5,6866,9656,0
6,1317,1766,0
7,9458,15439,1
8,12447,21216,0
9,5832,2363,1


#### Creating the final training table

In [76]:
# For each row (Source, Target), I add the corresponding degree, stored in either dfs or dft

df = df.merge(dfs, on = "Source", how = "left")
df = df.merge(dft, on = "Target", how = "left")
print(df)


#Turning it into numpy's arrays for training
labels = np.array(df["Link"])
df.drop(["Link"], axis = 1, inplace = True)
features = np.array(df)

print(labels)
print(features)

        Source  Target  Link  Degree_out  Degree_in
0        10481    5428     1         100         52
1         7353   30328     0           0          4
2         8627    3547     1         611         24
3        10232   21925     1         678          2
4         7110    3288     1          39         13
...        ...     ...   ...         ...        ...
453792   11186    4520     0           2        106
453793   12892   31446     0           5          0
453794   16857   23822     0           3          9
453795    5520    6394     1          12         35
453796   20318   28571     0           2          0

[453797 rows x 5 columns]
[1 0 1 ... 0 1 0]
[[10481  5428   100    52]
 [ 7353 30328     0     4]
 [ 8627  3547   611    24]
 ...
 [16857 23822     3     9]
 [ 5520  6394    12    35]
 [20318 28571     2     0]]


## Training

In [77]:
# To improve the training, increase n_estimators (number of trees in the forest)

rf = RandomForestRegressor(n_estimators = 20, random_state = 42)
t1 = time.time()
rf.fit(features, labels)
t2 = time.time()
print("Time : " + str(int(t2-t1)) + "s")

Time : 47s


## Testing DataSet

#### Loading DataSet + merging degrees

In [78]:
## I create my Test DataFrame df_test with the test file and rename my columns

df_test = pd.read_csv('testing.txt', sep=" ", header=None)
df_test.columns = ['Source', 'Target']

## I complete the DataFrame with the degree from the training set

df_test = df_test.merge(dfs, on = "Source", how = "left")
df_test = df_test.merge(dft, on = "Target", how = "left")

# As some Source or Target here have never been encountered in dfs or dft, the merging gives a NaN for the degree
# We replace each NaN by 0, without forgetting to recast the type to int
df_test = df_test.fillna(0).astype(int)
print(df_test)


        Source  Target  Degree_out  Degree_in
0          870   10284          11         22
1          620   15300         200         16
2        21115   31904           2          6
3         3021   28396           2          4
4        10780    6135          40          5
...        ...     ...         ...        ...
113445   13419   13751           3          4
113446   16696   20191           0          3
113447   10654   27692           5         12
113448    5409    1668         178         42
113449    1789     996           1         40

[113450 rows x 4 columns]


## Prediction

In [79]:
# Converting df_test into numpy array for prediction
test_features = np.array(df_test)

# As the predicted value is the mean of the predictions on the trees, it is not always 0 or 1
# We round the value to 0 or 1 and cast it into a int
test_predictions = np.round(rf.predict(test_features)).astype(int)

print(test_predictions)

# We format the DataFrame for the final export
df_pred = pd.DataFrame(test_predictions, columns = ["predicted"])
print(df_pred)

df_pred.to_csv("pred.csv", index_label = "id")

[1 1 0 ... 0 1 0]
        predicted
0               1
1               1
2               0
3               0
4               1
...           ...
113445          1
113446          0
113447          0
113448          1
113449          0

[113450 rows x 1 columns]
